In [2]:
import pandas as pd
import numpy as np
import os
from transformers import pipeline
import re




/home/ptyagi/miniconda3/envs/thesis_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-06 15:06:16.722111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733493976.737882 3021848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733493976.742714 3021848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 15:06:16.763020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

In [3]:
replies = pd.read_csv('/work/ptyagi/ClimateVisions/Tweet_Replies/Isaac_replies/replies_translated.csv')

In [4]:
replies.head()

,Unnamed: 0,id,conversation_id,created_at,text
0,1,1120457284268756992,1120240313484959744,2019-04-22T22:40:08.000Z,exactly breaking the law this is what those wo...
1,2,1120263796952567808,1120240313484959744,2019-04-22T09:51:17.000Z,what is your comment
2,3,1120261379427115010,1120240313484959744,2019-04-22T09:41:40.000Z,in the end these are untalented lobbyists whos...
3,4,1120260145362214912,1120240313484959744,2019-04-22T09:36:46.000Z,who is going to believe this organisations thi...
4,5,1120258615678525440,1120240313484959744,2019-04-22T09:30:42.000Z,and i did give both amnety and hws statements ...


In [5]:
replies['created_at'] = pd.to_datetime(replies['created_at'])
replies.rename(columns={'text':'replies'}, inplace=True)

In [6]:
replies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3795245 entries, 0 to 3795244
Data columns (total 5 columns):
 #   Column           Dtype              
---  ------           -----              
 0   Unnamed: 0       int64              
 1   id               int64              
 2   conversation_id  int64              
 3   created_at       datetime64[ns, UTC]
 4   replies          object             
dtypes: datetime64[ns, UTC](1), int64(3), object(1)
memory usage: 144.8+ MB


In [7]:
replies.head()

,Unnamed: 0,id,conversation_id,created_at,replies
0,1,1120457284268756992,1120240313484959744,2019-04-22 22:40:08+00:00,exactly breaking the law this is what those wo...
1,2,1120263796952567808,1120240313484959744,2019-04-22 09:51:17+00:00,what is your comment
2,3,1120261379427115010,1120240313484959744,2019-04-22 09:41:40+00:00,in the end these are untalented lobbyists whos...
3,4,1120260145362214912,1120240313484959744,2019-04-22 09:36:46+00:00,who is going to believe this organisations thi...
4,5,1120258615678525440,1120240313484959744,2019-04-22 09:30:42+00:00,and i did give both amnety and hws statements ...


In [8]:
replies_aug_2019 = replies[(replies['created_at'].dt.month == 8) & (replies['created_at'].dt.year == 2019)].copy()

In [9]:
replies_aug_2019.drop(columns='Unnamed: 0', inplace=True)

In [10]:
replies_aug_2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74908 entries, 9 to 3523787
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               74908 non-null  int64              
 1   conversation_id  74908 non-null  int64              
 2   created_at       74908 non-null  datetime64[ns, UTC]
 3   replies          74818 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 2.9+ MB


In [11]:
replies_aug_2019.sort_values(by='created_at', ascending=True, inplace=True)

In [12]:
replies_aug_2019.head()

,id,conversation_id,created_at,replies
293133,1156716497592639493,1156715487503536128,2019-08-01 00:01:18+00:00,yes
293132,1156716596775260161,1156715487503536128,2019-08-01 00:01:42+00:00,yes
293810,1156717206648098817,1156521373419921408,2019-08-01 00:04:07+00:00,oat and soy for me
293131,1156717326743425029,1156715487503536128,2019-08-01 00:04:36+00:00,i bet not one candidate understands what the c...
2935339,1156717403419660288,1156270691181826049,2019-08-01 00:04:54+00:00,"That yes, yes... that what happened six decade..."


In [13]:
replies_aug_2019.tail()

,id,conversation_id,created_at,replies
2944733,1167947540399939584,1167940127273828352,2019-08-31 23:49:27+00:00,Look on the networks who is Mr. Soros!? He is ...
343302,1167947674261147651,1167833288515309569,2019-08-31 23:49:59+00:00,yes i was explaining this conceptually to some...
2944767,1167948608995393536,1167848901489176577,2019-08-31 23:53:42+00:00,They only have ideas to oppose and confront an...
341598,1167949238497501185,1167576458379825153,2019-08-31 23:56:12+00:00,you are doing the right thing this is by far t...
342282,1167950192160604160,1167458952268963841,2019-08-31 23:59:59+00:00,people can not remove bias from their opinions...


In [14]:
tweet_ids = replies_aug_2019['conversation_id'].unique()

In [15]:
len(tweet_ids)

11105

In [16]:
def extract_tweet_id(filename):
    match = re.search(r'txt_(\d+)_', filename)
    return match.group(1) if match else None

In [17]:
tweet_texts = {}
tweets_directory = '/work/ptyagi/ClimateVisions/Tweet_Text/2019/08_August'
extracted_tweet_ids = []

for filename in os.listdir(tweets_directory):
    tweet_id = extract_tweet_id(filename)
    
    if tweet_id:
        extracted_tweet_ids.append(int(tweet_id))
        file_path = os.path.join(tweets_directory, filename)
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                tweet_texts[int(tweet_id)] = file.read()

In [18]:
len(extracted_tweet_ids)

57330

In [19]:
len(tweet_texts)

57330

In [20]:
directory = '/work/ptyagi/ClimateVisions/Tweet_Text/2019/08_August'
file_count = sum(len(files) for _, _, files in os.walk(directory))

print(f"Number of files: {file_count}")

Number of files: 57330


In [21]:
tweet_replies_aug_2019 = replies_aug_2019.copy()

In [22]:
tweet_replies_aug_2019['tweet_text'] = replies_aug_2019['conversation_id'].map(tweet_texts)


In [23]:
tweet_replies_aug_2019

,id,conversation_id,created_at,replies,tweet_text
293133,1156716497592639493,1156715487503536128,2019-08-01 00:01:18+00:00,yes,NaN
293132,1156716596775260161,1156715487503536128,2019-08-01 00:01:42+00:00,yes,NaN
293810,1156717206648098817,1156521373419921408,2019-08-01 00:04:07+00:00,oat and soy for me,NaN
293131,1156717326743425029,1156715487503536128,2019-08-01 00:04:36+00:00,i bet not one candidate understands what the c...,NaN
2935339,1156717403419660288,1156270691181826049,2019-08-01 00:04:54+00:00,"That yes, yes... that what happened six decade...",NaN
...,...,...,...,...,...
2944733,1167947540399939584,1167940127273828352,2019-08-31 23:49:27+00:00,Look on the networks who is Mr. Soros!? He is ...,NaN
343302,1167947674261147651,1167833288515309569,2019-08-31 23:49:59+00:00,yes i was explaining this conceptually to some...,Don’t understand why someone would buy a car a...
2944767,1167948608995393536,1167848901489176577,2019-08-31 23:53:42+00:00,They only have ideas to oppose and confront an...,NaN
341598,1167949238497501185,1167576458379825153,2019-08-31 23:56:12+00:00,you are doing the right thing this is by far t...,"🇺🇦Ukraine,Lone #ClimateStrike in Kharkiv, at C..."


In [24]:
nan_count = tweet_replies_aug_2019['tweet_text'].isna().sum()

print(f"Number of NaN values in tweet text: {nan_count}")

Number of NaN values in tweet text: 21669


In [25]:
tweet_replies_aug_2019.dropna(subset=['tweet_text', 'replies'], inplace=True)

In [26]:
len(tweet_replies_aug_2019['conversation_id'].unique())

7769

In [27]:
tweet_replies_aug_2019.head()

,id,conversation_id,created_at,replies,tweet_text
296464,1156746119180902400,1156745986548670465,2019-08-01 01:59:00+00:00,oh my god it totally was,Quote of the night? Gillibrand gets laughs say...
296463,1156746144963280896,1156745986548670465,2019-08-01 01:59:06+00:00,before or after she does her hair,Quote of the night? Gillibrand gets laughs say...
296462,1156746151435149312,1156745986548670465,2019-08-01 01:59:08+00:00,we did that,Quote of the night? Gillibrand gets laughs say...
296461,1156746252362510337,1156745986548670465,2019-08-01 01:59:32+00:00,try exorcism,Quote of the night? Gillibrand gets laughs say...
296460,1156746475994529797,1156745986548670465,2019-08-01 02:00:25+00:00,i am very unsurprised that the times would lik...,Quote of the night? Gillibrand gets laughs say...


In [28]:
tweet_replies_aug_2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53152 entries, 296464 to 342282
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               53152 non-null  int64              
 1   conversation_id  53152 non-null  int64              
 2   created_at       53152 non-null  datetime64[ns, UTC]
 3   replies          53152 non-null  object             
 4   tweet_text       53152 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(2)
memory usage: 2.4+ MB


In [29]:
tweet_replies_aug_2019['tweet_text'] = tweet_replies_aug_2019['tweet_text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '[URL]', x, flags=re.MULTILINE))
tweet_replies_aug_2019['tweet_text'] = tweet_replies_aug_2019['tweet_text'].apply(lambda x: re.sub(r'@\w+', '[USER]', x))


In [30]:
tweet_replies_aug_2019['replies'] = tweet_replies_aug_2019['replies'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '[URL]', x, flags=re.MULTILINE))
tweet_replies_aug_2019['replies'] = tweet_replies_aug_2019['replies'].apply(lambda x: re.sub(r'@\w+', '[USER]', x))

In [ ]:
tweet_replies_aug_2019.query("conversation_id == 1092171401648369665")['replies'].to_list()

['In addition to the considerable amounts of electricity that are already being exported abroad, new gas-fired power plants in Germany have been shut down because the operators prefer to keep dirty lignite-fired power plants running.',
 'For future e-cars, charging stations in company car parks are ideal, where the cars can be charged with solar energy during the day between the start of work and the end of the day.']

In [ ]:
tweet_replies_aug_2019.query("conversation_id == 1092171401648369665")['tweet_text'].to_list()

['Also. Frage an die #Grünsozialisten. Wenn 2018 D Netto Stromexporteur war und jetzt von 2020-2030 sowohl #Kohle als Kernstrom Weg fällt, woher beziehen wir dann den Strom für den obligatorischen #Tesla ?#Klimawandel #climatechange #climatestrike #FridaysForFuture [URL]',
 'Also. Frage an die #Grünsozialisten. Wenn 2018 D Netto Stromexporteur war und jetzt von 2020-2030 sowohl #Kohle als Kernstrom Weg fällt, woher beziehen wir dann den Strom für den obligatorischen #Tesla ?#Klimawandel #climatechange #climatestrike #FridaysForFuture [URL]']

In [33]:
tweet_replies_aug_2019.to_csv('/work/ptyagi/masterthesis/data/tmp/tweet_replies_aug_2019.csv', index=False)

In [37]:
classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", device=7)

In [38]:
predictions = classifier(tweet_replies_aug_2019['tweet_text'].tolist(), batch_size=8)

In [39]:
predictions[:10] # type: ignore

[{'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175},
 {'label': 'en', 'score': 0.8360907435417175}]

In [40]:
sorted(predictions, key=lambda x: x['score'])[:10] # type: ignore

[{'label': 'ru', 'score': 0.07305407524108887},
 {'label': 'ru', 'score': 0.07305407524108887},
 {'label': 'ru', 'score': 0.07305407524108887},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.07530135661363602},
 {'label': 'nl', 'score': 0.0753013864159584}]

In [41]:
tweet_replies_aug_2019['tweet_lang'] = [pred['label'] for pred in predictions] # type: ignore
tweet_replies_aug_2019['lang_confidence'] = [pred['score'] for pred in predictions] # type: ignore

In [42]:
tweet_replies_aug_2019.head()

,id,conversation_id,created_at,replies,tweet_text,tweet_lang,lang_confidence
296464,1156746119180902400,1156745986548670465,2019-08-01 01:59:00+00:00,oh my god it totally was,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296463,1156746144963280896,1156745986548670465,2019-08-01 01:59:06+00:00,before or after she does her hair,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296462,1156746151435149312,1156745986548670465,2019-08-01 01:59:08+00:00,we did that,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296461,1156746252362510337,1156745986548670465,2019-08-01 01:59:32+00:00,try exorcism,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296460,1156746475994529797,1156745986548670465,2019-08-01 02:00:25+00:00,i am very unsurprised that the times would lik...,Quote of the night? Gillibrand gets laughs say...,en,0.836091


In [43]:
tweet_replies_aug_2019_en = tweet_replies_aug_2019.query("tweet_lang == 'en'").copy()

In [44]:
tweet_replies_aug_2019_en.dropna(subset='replies', inplace=True)

In [45]:
tweet_replies_aug_2019_en.to_csv('/work/ptyagi/masterthesis/data/tmp/tweet_replies_aug_2019_en.csv', index=False)

In [46]:
tweet_replies_aug_2019_en

,id,conversation_id,created_at,replies,tweet_text,tweet_lang,lang_confidence
296464,1156746119180902400,1156745986548670465,2019-08-01 01:59:00+00:00,oh my god it totally was,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296463,1156746144963280896,1156745986548670465,2019-08-01 01:59:06+00:00,before or after she does her hair,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296462,1156746151435149312,1156745986548670465,2019-08-01 01:59:08+00:00,we did that,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296461,1156746252362510337,1156745986548670465,2019-08-01 01:59:32+00:00,try exorcism,Quote of the night? Gillibrand gets laughs say...,en,0.836091
296460,1156746475994529797,1156745986548670465,2019-08-01 02:00:25+00:00,i am very unsurprised that the times would lik...,Quote of the night? Gillibrand gets laughs say...,en,0.836091
...,...,...,...,...,...,...,...
343366,1167946521771307014,1167808357568323584,2019-08-31 23:45:24+00:00,i am no climate denier but i have plenty of co...,An argument I get from one #ClimateChange deni...,en,0.829527
341848,1167946714667372546,1167551088280686594,2019-08-31 23:46:10+00:00,thank you we love you,From affordable housing to reliable public tra...,en,0.887934
343302,1167947674261147651,1167833288515309569,2019-08-31 23:49:59+00:00,yes i was explaining this conceptually to some...,Don’t understand why someone would buy a car a...,en,0.856149
341598,1167949238497501185,1167576458379825153,2019-08-31 23:56:12+00:00,you are doing the right thing this is by far t...,"🇺🇦Ukraine,Lone #ClimateStrike in Kharkiv, at C...",en,0.828081


In [47]:
len(tweet_replies_aug_2019_en['conversation_id'].unique())

7345